<a href="https://colab.research.google.com/github/abolima/face_recognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install cmake

In [0]:
!pip install dlib

In [2]:
!pip install face_recognition

  Using cached https://files.pythonhosted.org/packages/3f/ed/ad9a28042f373d4633fc8b49109b623597d6f193d3bbbef7780a5ee8eef2/face_recognition-1.2.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/35/8d/e4ddf60452e2fb1ce3164f774e68968b3f110f1cb4cd353235d56875799e/dlib-19.16.0.tar.gz
  Running setup.py bdist_wheel for dlib: started
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Running setup.py bdist_wheel for dlib: still running...
  Run

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
torchvision 0.2.1 requires torch, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [11]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import face_recognition
import cv2 #OpenCV
from os import listdir

In [0]:
facecascade = cv2.CascadeClassifier('E:/haarcascade_frontalface_default.xml')

In [0]:
import matplotlib.pyplot as plt


In [59]:
#put your employee pictures in this path as name_of_employee.jpg
employee_pictures = "E:/photosdataset"

employees = dict()
known_face_encodings =[]
known_face_names =[]

for file in listdir(employee_pictures):
    employee, extension = file.split(".")
    img = face_recognition.load_image_file('E:\photosdataset\%s.jpg' % (employee))
    img_encoding = face_recognition.face_encodings(img)[0]
    known_face_encodings.append(img_encoding)
    known_face_names.append(employee)
    
print("employee representations retrieved successfully")

employee representations retrieved successfully


In [0]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
video_capture = cv2.VideoCapture(0)
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()